# Days Since

## Table of Contents
- [Prepare DataFrame](#Prepare-DataFrame)
    - [Read the CSV data from the file into the DataFrame](#Read-the-CSV-data-from-the-file-into-the-DataFrame)
    - [Define given date format](#Define-given-date-format)
    - [Count number of rows in the DataSet](#Count-number-of-rows-in-the-DataSet)
- [Determine day one](#Determine-day-one)
- [Function to provide "rata die"](#Function-to-provide-rata-die)
- [Iterate thru each row and update "rata die"](#Iterate-thru-each-row-and-update-rata-die)
- [Save DataFrame to CSV file](#Save-DataFrame-to-CSV-file)

# Prepare DataFrame

## Read the CSV data from the file into the DataFrame

In [1]:
using DataFrames, CSV, Dates
df = CSV.read("market_indicators.csv")
typeof(df)

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `using DataFrames; CSV.read(input, DataFrame; kw...)
│   caller = read(::String) at CSV.jl:40
└ @ CSV /Users/uki/.julia/packages/CSV/MKemC/src/CSV.jl:40


DataFrame

In [2]:
first(df,6)

,Day,Date,ISM Manufacturing,GDP Growth q/q % (Annualised),S&P 500,S&P 500 y/y
,Int64,Date,Float64?,Float64?,Float64?,String?
1,0,1948-03-31,43.3,0.062,missing,missing
2,30,1948-04-30,45.4,0.062,missing,missing
3,61,1948-05-31,49.5,0.062,missing,missing
4,91,1948-06-30,53.0,0.068,missing,missing
5,122,1948-07-31,48.4,0.068,missing,missing
6,153,1948-08-31,45.1,0.068,missing,missing


In [3]:
last(df,6)

,Day,Date,ISM Manufacturing,GDP Growth q/q % (Annualised),S&P 500,S&P 500 y/y
,Int64,Date,Float64?,Float64?,Float64?,String?
1,26207,2019-12-31,missing,0.021,missing,missing
2,26208,2020-01-01,50.9,missing,3225.52,19.28%
3,26238,2020-01-31,missing,missing,missing,missing
4,26239,2020-02-01,50.1,missing,2954.22,6.10%
5,26267,2020-02-29,50.1,missing,missing,missing
6,26298,2020-03-31,49.1,missing,missing,missing


In [4]:
date_column = 2

2

## Count number of rows in the DataSet

In [5]:
rows = size(df)[1]

1623

# Define given date format

In [6]:
value = df[1, date_column] # row 1, column 2
date_format = "yy-mm-dd" # e.g. "1950-05-01"
Dates.Date("1950-05-01", date_format) # == "1950-05-01"

1950-05-01

# Determine day one

In [7]:
day_one = Dates.datetime2rata(value) # typeof(day_one) = Int64

711217

# Function to provide rata die

- https://en.wikipedia.org/wiki/Rata_Die

In [8]:
value = "1948-04-30"
#date = Dates.Date(value, date_format) # no need
date = df[2,2] # second row, date
typeof(date) # Julia auto-conversion of "1948-04-30" to Date

Date

In [9]:
rata_die = Dates.datetime2rata(date) # typeof(rate_die) = Int64

711247

In [10]:
rata_die - day_one

30

In [11]:
function days_since_day_one(df, row, date_column) 
    date = df[row, date_column] # type Date
    #date = Dates.Date(value, date_format) # no need
    rata_die = Dates.datetime2rata(date) 
    rata_die = rata_die - day_one
    return rata_die
end

# TEST the function
days_since_day_one(df, 2, date_column) 

30

# Iterate thru each row and update rata die

In [12]:
column_days = 1
rows = size(df)[1] # first part of the returned tupple

for row = 1:rows
    days = days_since_day_one(df, row, date_column)
    # println(days)
    df[row, column_days] = days
end

In [13]:
first(df,6)

,Day,Date,ISM Manufacturing,GDP Growth q/q % (Annualised),S&P 500,S&P 500 y/y
,Int64,Date,Float64?,Float64?,Float64?,String?
1,0,1948-03-31,43.3,0.062,missing,missing
2,30,1948-04-30,45.4,0.062,missing,missing
3,61,1948-05-31,49.5,0.062,missing,missing
4,91,1948-06-30,53.0,0.068,missing,missing
5,122,1948-07-31,48.4,0.068,missing,missing
6,153,1948-08-31,45.1,0.068,missing,missing


In [14]:
last(df,20)

,Day,Date,ISM Manufacturing,GDP Growth q/q % (Annualised),S&P 500,S&P 500 y/y
,Int64,Date,Float64?,Float64?,Float64?,String?
1,25963,2019-05-01,51.6,missing,2941.76,8.22%
2,25993,2019-05-31,missing,missing,missing,missing
3,26023,2019-06-30,missing,0.02,missing,missing
4,26024,2019-07-01,51.3,missing,2980.38,5.83%
5,26054,2019-07-31,missing,missing,missing,missing
6,26055,2019-08-01,48.8,missing,2926.46,0.86%
7,26085,2019-08-31,missing,missing,missing,missing
8,26086,2019-09-01,48.2,missing,2976.74,2.15%
9,26115,2019-09-30,missing,0.021,missing,missing


[back to top](#Table-of-Contents)
<hr/>

# Save DataFrame to CSV file

In [15]:
CSV.write("market_indicators.csv", df, delim=',', header=true)

"market_indicators.csv"